# reproduce the yang zhang paper (1104.2487v2) on weighting for magnification bias

We need a lot of averages for this, all with respect to magnitude, in particular we need:

$\alpha-1$
$b_g$ and
$W$

$W$ itself requires the same averages.

In [1]:
# from magmod import shotnoise, C_l_DM_CAMB, bgal_new, sg
from magmod import *
from magbias_experiments import SKA_zhangpen, CLAR_zhangpen, SKA1, SKA2, cb_hirax as hirax, hirax512, LSST, LSST_nosgfit, n



#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$ 
Rescaling the galaxy number density by a factor of 104185.986576 to match the gold sample with 6330073646.61 total galaxies 
#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$ 



bgal_new has the same magnitude dependence as sg, ONLY ON THE BIN CENTER

shotnoise for the magnitude bin is straight forward

C_DM does not depend on magnitude

In [8]:
def Ngal_in_bin(zmin, mtab_edges, galsurv, NINT = 2000, maxmag = 27):
    """mtab_edges is magnitude bin edges, 
    upper means higher number and fainter, thus more galaxies"""

    zmax = zbg_max(maxmag, galsurv)
    z_integrate = np.linspace(zmin,zmax, NINT)


    dNdztab = np.array([nofz(z_integrate, mmm) for mmm in mtab_edges])
    dNdzdOm = np.trapz(dNdztab, z_integrate, axis = 1)
    

    
    #now subtract lower from upper
    Nz = (dNdzdOm[1:] - dNdzdOm[:-1]) * 4 * np.pi #already in rad!!!! and multiplying with all sky
    Nz = np.atleast_1d(Nz)
    if (Nz<0).any():
#         print Nz
        raise ValueError("cannot have negative number of galaxies, most likely we confused magnitudes here")
    return Nz



# def Ngal_in_bin_old(zmin, zmax, m_lower, m_upper, galsurv, NINT = 2000):
#     """m_lower and m_upper ar the lower and upper magnitude bin edges, 
#     upper means higher number and fainter, thus more galaxies"""


#     z_integrate = np.linspace(zmin,zmax, NINT)


#     dNdztab_lower = nofz(z_integrate, m_lower)
#     dNdztab_upper = nofz(z_integrate, m_upper)
    
    
#     dNzdOm_lower = np.trapz(dNdztab_lower, z_integrate)
#     dNzdOm_upper = np.trapz(dNdztab_upper, z_integrate)
    
# #     print dNzdOm_upper - dNzdOm_lower

    
#     #now subtract lower from upper
#     Nz = (dNzdOm_upper - dNzdOm_lower) * 4 * np.pi #already in rad!!!! and multiplying with all sky
#     Nz = np.atleast_1d(Nz)
#     if (Nz<0).any():
# #         print Nz
#         raise ValueError("cannot have negative number of galaxies, most likely we confused magnitudes here")
#     return Nz

def shotnoise_in_bin(zmin, mtab_edges, galsurv, maxmag = 27, NINT = 2000):
    return 4*pi / Ngal_in_bin(zmin, mtab_edges, galsurv, NINT, maxmag = maxmag)

# def shotnoise_in_bin(zmin, zmax, m_lower, m_upper, galsurv, NINT = 2000):
#     """m_lower and m_upper ar the lower and upper magnitude bin edges, 
#     upper means higher number and fainter, thus more galaxies"""


#     z_integrate = np.linspace(zmin,zmax, NINT)


#     dNdztab_lower = nofz(z_integrate, m_lower)
#     dNdztab_upper = nofz(z_integrate, m_upper)
    
    
#     dNzdOm_lower = np.trapz(dNdztab_lower, z_integrate)
#     dNzdOm_upper = np.trapz(dNdztab_upper, z_integrate)
    
# #     print dNzdOm_upper - dNzdOm_lower

    
#     #now subtract lower from upper
#     Nz = (dNzdOm_upper - dNzdOm_lower) * 4 * np.pi #already in rad!!!! and multiplying with all sky
#     Nz = np.atleast_1d(Nz)
#     if (Nz<0).any():
# #         print Nz
#         raise ValueError("cannot have negative number of galaxies, most likely we confused magnitudes here")
#     return 4*pi/Nz #shot noise! The 4 pi could be cancelled but this way I can check more easily that it's correct...




In [50]:
def alphaminusone(z, maxmag, exp = LSST): #2(alpha-1) = 5sg-2
    maxmag = np.atleast_1d(maxmag)
    res = np.array( [ 5*sg(z, exp, mmm)[0] - 2 for mmm in maxmag])
    return res

def average_no_W(A,B,N):
    """A,B are the arrays to be averaged, same length as N which is the bg number density of galaxies"""
    return np.sum(A*B*N, axis = 1)/np.sum(N) #A and B are matrices, ell x mag



def W_weight(C_DMtab, Cshottab, alpha_m_one_tab, bgal_tab, Ngal_tab, exp = LSST):
    """the optimal, scale dependent weight, eq. 10
    input: background redshift z, magnitude bin center mag (later will be full array, now just needs to be inside the table used for bgal_tab),
    array (for ell values) of dark matter power spectrum and the shot noise (number). 
    Then there are tables of alpha-1, bgal, Cshot and Ngal on a given magnitude binning (the binning itself not needed here)"""

    DM_by_shot = np.outer(C_DMtab,1/Cshottab) #this need to have right shape, as C_DM is length ell and Cshot is length magtab
    num = -average_no_W(alpha_m_one_tab, bgal_tab, Ngal_tab) * DM_by_shot
    denom = 1 + average_no_W(bgal_tab,bgal_tab, Ngal_tab) * DM_by_shot
    fac1 = num/denom

    
#     return np.outer( np.ones( len(C_DMtab)),alpha_m_one_tab) + bgal_tab * fac1 #outer product to get right shape
    return alpha_m_one_tab + bgal_tab * fac1 #outer product to get right shape



def W_weight_singlebin(z, mag, C_DMtab, Cshottt, alpha_m_one_tab, bgal_tab, Ngal_tab, exp = LSST):
    """the optimal, scale dependent weight, eq. 10
    input: background redshift z, magnitude bin center mag (later will be full array, now just needs to be inside the table used for bgal_tab),
    array (for ell values) of dark matter power spectrum and the shot noise (number). 
    Then there are tables of alpha-1, bgal, Cshot and Ngal on a given magnitude binning (the binning itself not needed here)"""

    DM_by_shot = C_DMtab/Cshottt #this need to have right shape, as C_DM is length ell and Cshot is length magtab
    
    num = -average_no_W(alpha_m_one_tab, bgal_tab, Ngal_tab) * DM_by_shot
    denom = 1 + average_no_W(bgal_tab,bgal_tab, Ngal_tab) * DM_by_shot
    fac1 = num/denom
    bgalll = bgal_new(z, mag, experiment = exp)
    almino = alphaminusone(z, mag, exp = exp)
    
    
    print bgalll, "bg"
    print almino, "alpha minus one"
    print Cshottt, "Cshot"
    
    return almino + fac1 * bgalll
    
    

# def average_W(A,B,N,weights):
#     """average weighted by weights, important: weights will be matrix (ell x mags), thus A and be also need same shape, 
#     and N need length mags."""
#     W_ar = W_weight(mags)
#     return average_no_W(A,B,N*weights)


testing things:

In [20]:
# B1 1st bin:
zflow = 0.34127101; zfhigh = 0.84127101; zfmean = (zfhigh+zflow)/2; delta_zf = (zfhigh-zflow)/2

#background z:
buffer_z = 0.1
zlow = zfhigh + buffer_z
#zhigh should not be needed

#small mag range:
magmin = 25; magmax = 27; Nmag = 3
magtab_edges = np.linspace(magmin, magmax, Nmag)
magtab_centers = (magtab_edges[1:] + magtab_edges[:-1])/2

mag_test = magtab_centers[1]
magmin_test = magtab_edges[1]; magmax_test = magtab_edges[2]


#small ell range
SKAarea = SKA['S_area']
# lminSKA = np.amax([10,np.int(np.around(2*pi/np.sqrt(SKAarea)))]) #never use lmin <20
lminSKA = 200
print "WARNING: lminSKA is set to large value {}!".format(lminSKA)
lend = 2200;
ltabSKA = np.arange(lminSKA, lend + lminSKA, lminSKA)


In [11]:
C_DM_tab = C_l_DM_CAMB(ltabSKA, zlow, galsurv = LSST)
Cshot_test = shotnoise_in_bin(zlow, magtab_edges, LSST)

In [13]:
alphaminusone_tab = alphaminusone(zlow, magtab_centers) #actually a mean redshift should be used
bias_g_tab = np.array([bgal_new(zlow, mmm) for mmm in magtab_centers])
N_g_tab = Ngal_in_bin(zlow,magtab_edges, LSST)

It is unclear to me what mean redshift to use for the background galaxies. Because here alpha-1 and bg are not integrated over, but evaluated at a single redshift...

In [15]:
Wtest = W_weight(C_DM_tab, Cshot_test, alphaminusone_tab, bias_g_tab, N_g_tab)

In [16]:
print Wtest

[[ 0.19355644 -0.16345818]
 [ 0.12752405 -0.20259644]
 [ 0.05578155 -0.25022057]
 [-0.01560434 -0.30390536]
 [-0.0749769  -0.35429278]
 [-0.12394613 -0.40049931]
 [-0.16415348 -0.44208213]
 [-0.19592445 -0.47757745]
 [-0.22173716 -0.50832285]
 [-0.24269249 -0.53465606]
 [-0.25985189 -0.55720605]]


Assuming that the weighting function works, we also need the interferometer and dish noise, the foreground power spectrum and C_HI-mu. For the last one we use eq. 6 to relate to Cl_HIxmag.

Do not forget: galaxy background, HI foreground

In [21]:
C_HIHI_tab = C_l_HIHI_CAMB(ltabSKA, zflow, zfhigh)
C_noise_fg_tab = noise_cls_single_dish(ltabSKA, ztonu21(zfmean), SKA1, 256) * np.ones( len(ltabSKA) )
C_HIxmag_tab = np.array( [Cl_HIxmag_CAMB(ltabSKA, zfmean, delta_zf, zlow, MAXMAG = mmm)
                          for mmm in magtab_centers]).T #transpose to match shape

(2, 11)


In [69]:
def S2N_weighted(ltab, deltaell, fsky, Cfg, CSfg, CSbg, CHImu, biasg, Weight, Ngal):
    num = (2*ltab + 1) * deltaell * fsky
    denom = 1 + (Cfg + CSfg) * ( 
        average_no_W(biasg, Weight, Ngal)**2 * C_DM_tab + 
        average_no_W(Weight**2, CSbg, Ngal)) / (average_no_W(CHImu, Weight, Ngal)**2) #I put the shot noise into the average
#         average_no_W(Weight, Weight, Ngal) * CSbg) / (average_no_W(CHImu, Weight, Ngal)**2)
    frac = num/denom
    res = np.sum(frac, axis = 0)
    return res

In [70]:
deltaell = ltabSKA[1]-ltabSKA[0]
print "delta ell = {}".format(deltaell)
fsky = SKA1["S_area"] / (4*np.pi)

S2Ntest = S2N_weighted(ltabSKA, deltaell, fsky, C_HIHI_tab, C_noise_fg_tab, Cshot_test, C_HIxmag_tab, bias_g_tab, Wtest, N_g_tab)

delta ell = 200


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in true_divide
  """


In [71]:
print S2Ntest

17.586356750476753


this S2N is about a factor of 2 better than in our paper (fig 7 dark blue). But a few open questions remain to this approach:

-- we should put bg and alpha into the integrations, i.e. the power spectra into the averaging

-- shot noise NEEDS to be inside the averaging, typo/error in their paper???

-- I am honestly unsure what weighting with a non-positive, sign-flipping weight actually means, but I guess in our case it just corrects for negative signals (demag)?

-- we must keep in mind that we fixed (alpha - 1)bg ourselves, and this has a big impact here too.

-- I should check with smaller ∆ell!